In [1]:
import pandas as pd

In [2]:
weather_data_file = '../data/NOAA_weather_data/2005-2015_california_box_all.csv'
fire_data_file = '../data/USDA_wildfire_data/regridded_california_fires.csv'
training_data_file = '../data/training_data/2005-2015_training_data_box.csv'

# Parallelization options
N_PROCESSES = 15
JOBS_PER_PROCESS = 1

In [3]:
weather = pd.read_csv(weather_data_file)
weather['time'] = pd.to_datetime(weather['time'])
weather['date'] = weather['time'].dt.date
weather['date'] = pd.to_datetime(weather['date'])

fires = pd.read_csv(fire_data_file)
fires['ignition'] = 1
fires['date'] = pd.to_datetime(fires['date'])

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44669040 entries, 0 to 44669039
Data columns (total 14 columns):
time        datetime64[ns]
lat         float64
lon         float64
air.sfc     float64
air.2m      float64
apcp        float64
crain       float64
rhum.2m     float64
dpt.2m      float64
pres.sfc    float64
uwnd.10m    float64
vwnd.10m    float64
veg         float64
date        datetime64[ns]
dtypes: datetime64[ns](2), float64(12)
memory usage: 4.7 GB


In [5]:
fires.head()

,lat,lon,date,time,size,size_class,ignition
0,39.93427,-121.1598,2005-02-02,1300.0,0.10,A,1
1,38.85770,-120.5208,2004-05-12,845.0,0.25,A,1
2,39.13992,-120.5874,2004-05-31,1921.0,0.10,A,1
3,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1
4,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1


In [6]:
fires.tail()

,lat,lon,date,time,size,size_class,ignition
189545,40.32918,-122.4153,2015-09-26,1726.0,0.01,A,1
189546,37.62744,-120.9710,2015-10-05,126.0,0.20,A,1
189547,37.62744,-120.9710,2015-05-02,2052.0,0.10,A,1
189548,37.62744,-120.9710,2015-10-14,2309.0,2.00,B,1
189549,34.15059,-116.7372,2015-03-14,2128.0,0.10,A,1


In [7]:
training_data = pd.merge(weather, fires, on=['lat', 'lon', 'date'], how='left')

In [8]:
training_data.head()

,time_x,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,date,time_y,size,size_class,ignition
0,2005-01-01,31.63322,-125.3449,289.18558,287.6236,0.0,0.0,81.03484,284.37213,101686.18,7.070458,-3.266892,0.0,2005-01-01,NaN,NaN,NaN,NaN
1,2005-01-01,31.90358,-125.4247,289.18558,287.6236,0.0,0.0,81.03484,284.37213,101686.18,7.492333,-2.751267,0.0,2005-01-01,NaN,NaN,NaN,NaN
2,2005-01-01,31.70029,-125.0272,289.24808,287.6236,0.0,0.0,79.84734,284.12994,101686.18,7.117333,-2.696579,0.0,2005-01-01,NaN,NaN,NaN,NaN
3,2005-01-01,31.97103,-125.1057,288.99808,287.5611,0.0,0.0,82.65984,284.61432,101686.18,7.507958,-2.063767,0.0,2005-01-01,NaN,NaN,NaN,NaN
4,2005-01-01,32.24205,-125.1849,288.81058,287.4361,0.0,0.0,82.65984,284.48150,101686.18,7.664208,-1.610642,0.0,2005-01-01,NaN,NaN,NaN,NaN


In [9]:
training_data['air.sfc'].isnull().sum()

0

In [10]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44802584 entries, 0 to 44802583
Data columns (total 18 columns):
time_x        datetime64[ns]
lat           float64
lon           float64
air.sfc       float64
air.2m        float64
apcp          float64
crain         float64
rhum.2m       float64
dpt.2m        float64
pres.sfc      float64
uwnd.10m      float64
vwnd.10m      float64
veg           float64
date          datetime64[ns]
time_y        float64
size          float64
size_class    object
ignition      float64
dtypes: datetime64[ns](2), float64(15), object(1)
memory usage: 6.3+ GB


In [11]:
training_data.rename(columns={'time_x':'weather_bin_time','time_y':'fire_discovery_time'}, inplace=True)
training_data.drop(['date'], axis=1, inplace=True)
training_data.head()

,weather_bin_time,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,size_class,ignition
0,2005-01-01,31.63322,-125.3449,289.18558,287.6236,0.0,0.0,81.03484,284.37213,101686.18,7.070458,-3.266892,0.0,NaN,NaN,NaN,NaN
1,2005-01-01,31.90358,-125.4247,289.18558,287.6236,0.0,0.0,81.03484,284.37213,101686.18,7.492333,-2.751267,0.0,NaN,NaN,NaN,NaN
2,2005-01-01,31.70029,-125.0272,289.24808,287.6236,0.0,0.0,79.84734,284.12994,101686.18,7.117333,-2.696579,0.0,NaN,NaN,NaN,NaN
3,2005-01-01,31.97103,-125.1057,288.99808,287.5611,0.0,0.0,82.65984,284.61432,101686.18,7.507958,-2.063767,0.0,NaN,NaN,NaN,NaN
4,2005-01-01,32.24205,-125.1849,288.81058,287.4361,0.0,0.0,82.65984,284.48150,101686.18,7.664208,-1.610642,0.0,NaN,NaN,NaN,NaN


In [12]:
training_data.describe()

,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,ignition
count,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,4.480258e+07,503968.000000,671296.000000,671296.0
mean,3.703931e+01,-1.193083e+02,2.878614e+02,2.869462e+02,1.056170e-01,5.184511e-02,5.966921e+01,2.766226e+02,9.266192e+04,1.654170e+00,-1.001125e+00,1.422232e+01,1388.990682,84.102874,1.0
std,3.307911e+00,3.560928e+00,1.196939e+01,9.167043e+00,6.497919e-01,2.217142e-01,2.941525e+01,8.844924e+00,8.519585e+03,2.919533e+00,4.210851e+00,1.980744e+01,496.087687,2512.417034,0.0
min,3.153068e+01,-1.254445e+02,2.387660e+02,2.446787e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.921248e+02,6.624355e+04,-2.030144e+01,-2.067075e+01,0.000000e+00,0.000000,0.001000,1.0
25%,3.414266e+01,-1.223950e+02,2.810404e+02,2.822150e+02,0.000000e+00,0.000000e+00,3.150000e+01,2.696379e+02,8.456665e+04,-3.768673e-01,-3.444094e+00,0.000000e+00,1122.000000,0.100000,1.0
50%,3.693702e+01,-1.193113e+02,2.874534e+02,2.870248e+02,0.000000e+00,0.000000e+00,6.409358e+01,2.770536e+02,9.386506e+04,1.595291e+00,-7.750001e-01,4.200000e+00,1430.000000,0.250000,1.0
75%,3.987430e+01,-1.162261e+02,2.920649e+02,2.911728e+02,7.812500e-03,0.000000e+00,8.959193e+01,2.845935e+02,1.014531e+05,3.534164e+00,1.861440e+00,2.130000e+01,1722.000000,1.000000,1.0
max,4.299324e+01,-1.131324e+02,3.384141e+02,3.229702e+02,4.197656e+01,1.000000e+00,1.000312e+02,3.044732e+02,1.041916e+05,2.268434e+01,2.987953e+01,8.370000e+01,2359.000000,315578.800000,1.0


In [ ]:
training_data.to_csv(training_data_file, index=False)